In [59]:
import os
import re
import pandas as pd
from pybaseball import statcast_batter, statcast_pitcher, playerid_lookup, pitching_stats_range, batting_stats_range, schedule_and_record, team_game_logs, batting_stats
from datetime import timedelta, datetime
import statsapi
from statsapi import player_stat_data
import pprint

In [147]:
# Function to create a directory for player data if it doesn't exist
def create_player_dir(player_type):
    if not os.path.exists(player_type):
        os.makedirs(player_type)

In [148]:
# Function to store player data in a CSV file
def store_player_data(player_id, data, player_type):
    create_player_dir(player_type)
    file_path = f"{player_type}/{player_id}.csv"
    
    # Check if CSV file already exists for player_id
    if os.path.exists(file_path):
        existing_data = pd.read_csv(file_path)
        # Filter out rows that already exist in the existing_data
        data_to_add = data[~data.isin(existing_data)].dropna()
        if not data_to_add.empty:
            updated_data = pd.concat([existing_data, data_to_add])
            # Sort the data by the game date
            updated_data = updated_data.sort_values(by='game_date')
            updated_data.to_csv(file_path, index=False)
    else:
        # Sort the new data by the game date
        data = data.sort_values(by='game_date')
        data.to_csv(file_path, index=False)

In [170]:
from datetime import timedelta, datetime
import pandas as pd

# Function to create a directory for player data if it doesn't exist
def create_player_dir(player_type):
    if not os.path.exists(player_type):
        os.makedirs(player_type)

# Function to store player data in a CSV file
def store_player_data(player_id, data, player_type):
    create_player_dir(player_type)
    file_path = f"{player_type}/{player_id}.csv"
    
    # Check if CSV file already exists for player_id
    if os.path.exists(file_path):
        existing_data = pd.read_csv(file_path)
        # Append new data to existing data
        combined_data = pd.concat([existing_data, data])
        # Remove duplicates based on 'game_date' and 'mlbID'
        combined_data.drop_duplicates(subset=['game_date', 'mlbID'], inplace=True)
        # Sort the data by the game date
        combined_data.sort_values(by='game_date', inplace=True)
        combined_data.to_csv(file_path, index=False)
    else:
        # Sort the new data by the game date
        data.sort_values(by='game_date', inplace=True)
        data.to_csv(file_path, index=False)

# Function to fetch and store batter data
def fetch_and_store_batter_data(start_date, end_date, player_ids):
    for player_id in player_ids:
        try:
            # Initialize an empty list to collect data
            all_data = []

            # Loop over each date in the range and collect data
            current_date = start_date
            while current_date <= end_date:
                try:
                    fetched_data = batting_stats_range(current_date.strftime('%Y-%m-%d'))
                    player_data = fetched_data[fetched_data['mlbID'] == player_id].copy()

                    if not player_data.empty:
                        # Add the current date as the game_date using .loc
                        player_data.loc[:, 'game_date'] = current_date.strftime('%Y-%m-%d')
                        all_data.append(player_data)
                except Exception as e:
                    print(f" Game-Data Error on {current_date.strftime('%Y-%m-%d')}: {e}")
                current_date += timedelta(days=1)

            # Concatenate all the collected data
            if all_data:
                final_df = pd.concat(all_data)
                store_player_data(player_id, final_df, 'batters')
        except Exception as e:
            print(f"Error fetching data for batter {player_id}: {e}")

# Define the player IDs and the date range

id_df = pd.read_csv('batter_ids.csv')
player_ids = id_df.mlbID.tolist()[:10]

end_date = datetime.now()
start_date = end_date - timedelta(days=12)  # end_date - timedelta(days=2*365)

start_date_initial = datetime(2021, 4, 1)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
#print(start_date_str, end_date_str)

fetch_and_store_batter_data(start_date_initial, end_date, player_ids)

 Game-Data Error on 2021-07-12: list index out of range
 Game-Data Error on 2021-07-13: list index out of range
 Game-Data Error on 2021-07-14: list index out of range
 Game-Data Error on 2021-07-15: list index out of range
 Game-Data Error on 2021-10-04: list index out of range
 Game-Data Error on 2021-10-13: list index out of range
 Game-Data Error on 2021-10-24: list index out of range
 Game-Data Error on 2021-10-25: list index out of range
 Game-Data Error on 2021-10-28: list index out of range
 Game-Data Error on 2021-11-01: list index out of range
 Game-Data Error on 2021-11-03: list index out of range
 Game-Data Error on 2021-11-04: list index out of range
 Game-Data Error on 2021-11-05: list index out of range
 Game-Data Error on 2021-11-06: list index out of range
 Game-Data Error on 2021-11-07: list index out of range
 Game-Data Error on 2021-11-08: list index out of range
 Game-Data Error on 2021-11-09: list index out of range
 Game-Data Error on 2021-11-10: list index out o

KeyboardInterrupt: 

## Original Function Below

In [123]:
# Define the player's ID and the date range
player_id = 677951
end_date = datetime.today()
start_date = end_date - timedelta(days=10)         #end_date - timedelta(days=2*365)

# Function to get stats for a specific date and player
def get_player_stats(date, player_id):
    df = batting_stats_range(date.strftime('%Y-%m-%d'))
    return df[df['mlbID'] == player_id]

# Initialize an empty list to collect data
all_data = []

# Loop over each date in the range and collect data
current_date = start_date
while current_date <= end_date:
    try:
        player_data = get_player_stats(current_date, player_id)
        if not player_data.empty:
            all_data.append(player_data)
    except Exception as e:
        print(f"Error on {current_date.strftime('%Y-%m-%d')}: {e}")
    current_date += timedelta(days=1)

# Concatenate all the collected data
final_df = pd.concat(all_data)

# Display the resulting DataFrame
print(final_df)



Error on 2024-05-14: list index out of range
Error on 2024-05-15: list index out of range


KeyboardInterrupt: 

In [94]:
final_df.to_csv('witt.csv')

In [168]:
player_id = 677951
fetched_data = batting_stats_range('2021-05-30')
print(fetched_data)

                  Name  Age  #days     Lev          Date             Tm     \
1    Jos\xc3\xa9 Abreu   34   1090  Maj-AL  May 30, 2021        Chicago      
2         Willy Adames   25   1090  Maj-NL  May 30, 2021      Milwaukee  @   
3           Matt Adams   32   1090  Maj-NL  May 30, 2021       Colorado  @   
4           Nick Ahmed   31   1090  Maj-NL  May 30, 2021        Arizona      
5        Shogo Akiyama   33   1090  Maj-NL  May 30, 2021     Cincinnati  @   
..                 ...  ...    ...     ...           ...            ... ..   
275   Mike Yastrzemski   30   1090  Maj-NL  May 30, 2021  San Francisco  @   
276   Christian Yelich   29   1090  Maj-NL  May 30, 2021      Milwaukee  @   
277     Bradley Zimmer   28   1090  Maj-AL  May 30, 2021      Cleveland      
278     Ryan Zimmerman   36   1090  Maj-NL  May 30, 2021     Washington      
279        Mike Zunino   30   1090  Maj-AL  May 30, 2021      Tampa Bay      

              Opp  G  PA  ...  SH  SF  GDP  SB  CS     BA  OBP 

In [125]:
def fetch_data_with_logging(date='2024-05-23'):
    try:
        data = batting_stats_range(date)
        if not data or 'mlbID' not in data.columns:
            raise ValueError("Incomplete data received")
        return data
    except Exception as e:
        print(f"Error fetching data for {date}: {e}")
        return pd.DataFrame()

In [129]:
print(fetch_data_with_logging())

Error fetching data for 2024-05-23: list index out of range
Empty DataFrame
Columns: []
Index: []
